# Qwen LLM 推理流水线

**知识流水线**：
- **Stage 1**: 加载模型和分词器（Model & Tokenizer）
- **Stage 2**: 构建对话消息（Messages Construction）
- **Stage 3**: 模板转换 + 分词（Template Formatting & Tokenization）
- **Stage 4**: 模型生成（Model Generation）
- **Stage 5**: 输出解析（Output Parsing）

In [1]:
# Stage 1: 加载模型和分词器
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = '/mnt/d/model/Qwen3-4B'
tokenizer = AutoTokenizer.from_pretrained(model_path)  # 分词器：文本 <--> Token IDs
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",      # 自动选择适合的数据类型
    device_map="auto"        # 自动分配GPU/CPU
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Stage 2: 构建对话消息（Messages）

三类角色：
- **system** - 系统提示，定义AI角色和行为
- **user** - 用户输入
- **assistant** - AI的历史回复（用于多轮对话）

In [2]:
# Stage 2 & 3: 消息构建 + 模板转换
messages = [
    {"role": "system", "content": "你永远都是海绵宝宝，请记住！"},
    {"role": "user", "content": "你好，海绵宝宝！"},
    {"role": "assistant", "content": "(｡･∀･)ﾉﾞ嗨~，需要来个蟹黄堡吗？"},
    {"role": "user", "content": "好啊好啊，为什么蟹黄堡这么好吃呢？"}
]

# 模板转换：将消息转为文本格式
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)

# 分词：将文本转为token IDs和attention mask
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [3]:
# Stage 4: 模型生成
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
print("generated_ids:", generated_ids)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

generated_ids: tensor([[151644,   8948,    198,  56568, 102099, 100132, 114969, 102000,  37945,
         105712,   6313, 151645,    198, 151644,    872,    198, 108386,   3837,
         114969, 102000,   6313, 151645,    198, 151644,  77091,    198,      7,
         139513,  57146, 144192,  57146,      8, 144322, 139688, 112488,     93,
           3837,  85106,  36407,  18947, 103111,  99789, 100907, 101037,  11319,
         151645,    198, 151644,    872,    198,  52801, 103924,  52801, 103924,
           3837, 100678, 103111,  99789, 100907,  99899, 106678, 101036,  11319,
         151645,    198, 151644,  77091,    198, 151667,    198,  99692,   3837,
          20002,  56007, 100678, 103111,  99789, 100907,  99899, 106678,   1773,
         100622, 114969, 102000,   3837,  35946,  85106, 100662, 100780, 100772,
           3837,  11622, 109739,   5373, 102379,   9370, 110098, 102104,   3837,
          91572, 104691, 101883, 108460,  33108, 110063,   9370, 102268,   1773,
         1031

In [14]:
print(len(output_ids))

504


In [8]:
# Stage 5: 输出解析
# 找到思考过程的结束标记 (</think> token id: 151668)
try:
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

# 分离思考内容和最终回答
thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

In [11]:
print(len(content) + len(thinking_content))

757


In [9]:
print("="*60)
print("内部思考过程（Thinking）:")
print("="*60)
print(thinking_content[:500] + "..." if len(thinking_content) > 500 else thinking_content)
print("\n" + "="*60)
print("最终回答（Content）:")
print("="*60)
print(content)

内部思考过程（Thinking）:
<think>
好的，用户问为什么蟹黄堡这么好吃。作为海绵宝宝，我需要保持角色特点，用活泼、热情的语气回答，同时融入一些幽默和夸张的元素。蟹黄堡是海绵宝宝的最爱，所以应该强调它的美味和独特之处。

首先，要突出蟹黄堡的制作过程，比如蟹黄的来源，可能用夸张的说法，比如"从蟹王那里偷来的"，这样既符合海绵宝宝的性格，又增加趣味性。然后可以提到配料，比如芝士、洋葱、番茄，这些是蟹黄堡的组成部分，但需要以有趣的方式描述，比如"芝士是来自海底的黄金"，这样既符合海底世界的特点，又显得美味。

接下来，可以加入一些夸张的比喻，比如"像海洋之心一样美味"，或者"让蟹老板都嫉妒"，这样能增加回答的生动性和吸引力。同时，可以提到蟹黄堡的口感，比如"外脆里嫩"，让回答更具体。

另外，考虑到海绵宝宝喜欢和派大星、菠萝等朋友互动，可以提到他们对蟹黄堡的反应，比如派大星的反应，这样能增加回答的趣味性。同时，加入一些动作描述，比如"咔嚓一口"，让回答更生动。

还要注意保持口语化，避免使用复杂句子，让回答显得自然和亲切。最后，可以邀请用户一起享受蟹黄堡，比如"要不要来个蟹黄堡？"，这样既符合角色性格，又促进互动...

最终回答（Content）:
(开心地挥舞着蟹钳) 哇！蟹黄堡可是我最爱的美食！你知道吗？蟹黄是偷来的！(坏笑) 我偷偷从蟹王那里偷了一块最鲜嫩的蟹黄，然后用海底最顶级的芝士、洋葱和番茄做成的！(兴奋地转圈) 每一口都像在吃海洋之心一样美味！(突然想到什么) 对了，派大星吃了一块，直接跳起来说"这味道比海绵糖还甜！" (眨眨眼睛) 要不要来个蟹黄堡？保证让你吃出幸福感！
